## Install Libraries

One-click install of all the required libraries


In [13]:
!pip install -qU langchain langchainhub langchain-community langchain-fireworks langchain-huggingface langchain-mongodb arxiv pymupdf datasets pymongo

## Setup Pre-requisites

You should have obtained the MongoDB connection string and Fireworks API Key during the first hands-on breakout. Set those here when prompted.


In [2]:
import getpass
import os

MONGODB_URI = getpass.getpass("Enter your MongoDB connection string:")

In [3]:
os.environ["FIREWORKS_API_KEY"] = getpass.getpass("Enter Fireworks API key:")

## Ingest Data into MongoDB Atlas

We will use MongoDB Atlas as the vector store/knowledge base for one of the agent tools. But first, we need to create the knowledge base.


In [ ]:
# Download a dataset consisting of a subset of ArxiV papers along with their embedded abstracts.
# Available at https://huggingface.co/datasets/mongodb-eai/arxiv-embeddings

import pandas as pd
from datasets import load_dataset

data = load_dataset("mongodb-eai/arxiv-embeddings")
dataset_df = pd.DataFrame(data["train"])

In [ ]:
# Preview the dataset
dataset_df.head()

In [6]:
# Initialize MongoDB client and other variables for the vector store

from pymongo import MongoClient

# Reference docs: https://pymongo.readthedocs.io/en/stable/api/pymongo/mongo_client.html
# Refer to the docs above to initialize a MongoDB Python client
client = INSERT CODE HERE

# Name of the database -- Change if needed or leave as is
DB_NAME = "agents_workshop"
# Name of the collection -- Change if needed or leave as is
COLLECTION_NAME = "knowledge"
# Name of the vector search index -- Change if needed or leave as is
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"
collection = client[DB_NAME][COLLECTION_NAME]

In [ ]:
# Delete existing documents from collection
# Reference docs: https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html
# Refer to the docs above to bulk delete all existing records in the collection defined above -- should be a one-liner
INSERT CODE HERE

# Ingest data into the collection
records = dataset_df.to_dict("records")

# Insert data into collection
# Reference docs: https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html
# Refer to the docs above to bulk insert multiple records into the collection defined above -- should be a one-liner
INSERT CODE HERE
print("Data ingestion into MongoDB completed")

## Create Vector Search Index Defintion

Define a vector search index on the collection created above with the following index definition. To be done in the Atlas UI. Follow the documentation for instructions.

```
{
  "fields": [
    {
      "type": "vector",
      "path": "embedding",
      "numDimensions": 1024,
      "similarity": "cosine"
    }
  ]
}
```


## Create MongoDB Vector Store Retriever

Now that we have ingested data into a MongoDB Collection, let's use it to create a vector store retriever.


In [ ]:
# Embedding model to use for the vector store -- DO NOT CHANGE
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

In [ ]:
from langchain_mongodb import MongoDBAtlasVectorSearch

# Create a MongoDBAtlas vector store
# Reference docs: https://api.python.langchain.com/en/latest/_modules/langchain_mongodb/vectorstores.html#MongoDBAtlasVectorSearch
# Use the `from_connection_string` method of the MongoDBAtlAsVectorSearch class.
# Arguments: connection_string, namespace, embedding, index_name, text_key
vector_store = INSERT CODE HERE

# Construct a retriever from the vector store
# Reference docs: https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/vectorstore/
# Set the search type for the retriever to `similarity` and `k` to 5.
retriever = INSERT CODE HERE

## Instantiate Chat Completion LLM

Instantiate the chat completion LLM to use as the "brain" of our agent and for any of the tools if required.

We will use Fireworks AI's open-source AND free `firefunction-v1` model.


In [11]:
from langchain_fireworks import ChatFireworks

# Reference docs: https://python.langchain.com/v0.1/docs/integrations/chat/fireworks/
# Create an instance of `ChatFireworks` with the model `accounts/fireworks/models/firefunction-v1`, temperature set to 0.0 and max tokens set to 1024
llm = INSERT CODE HERE

## Create Agent Tools

Define tools for the agent to use.


In [14]:
from langchain.tools import tool
from langchain_community.document_loaders import ArxivLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


@tool
def get_paper_metadata_from_arxiv(topic: str) -> list:
    """
    Fetch and return paper metadata for 10 arxiv papers matching the given topic, for example: Retrieval Augmented Generation.

    Args:
    topic (str): The topic to find papers for on arXiv.

    Returns:
    list: Metadata about the papers matching the topic.
    """
    docs = ArxivLoader(query=topic, load_max_docs=5).load()
    # Extract just the metadata from each document
    metadata = [doc.metadata for doc in docs]
    return metadata


@tool
def get_paper_summary_from_arxiv(id: str) -> str:
    """
    Fetch and return the summary for a single research paper from arXiv given the paper ID, for example: 1605.08386.

    Args:
    id (str): The paper ID.

    Returns:
    str: Summary of the paper.
    """
    # Reference docs:
    # https://python.langchain.com/v0.1/docs/integrations/document_loaders/arxiv/
    # https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.arxiv.ArxivLoader.html
    # Create a tool that uses the ArxivLoader to return summaries given a paper ID (`id`).
    # HINT: Use the `get_summaries_as_docs` method of `ArxivLoader`
    # Handle the case where the paper ID is invalid i.e. number of docs returned from `ArxivLoader` are 0.
    INSERT CODE HERE


@tool
def answer_questions_about_topics(query: str) -> list:
    """
    Answer questions about a given topic based on information in the knowledge base.

    Args:
    query (str): User query about a topic.

    Returns:
    str: Information about the topic.
    """
    # Reference docs: https://python.langchain.com/v0.1/docs/use_cases/question_answering/quickstart/
    # Follow the example in the reference docs above to create a tool that uses the Fireworks LLM we instantiated above to answer questions based on content in our MongoDB Atlas knowledge base.
    # Create a RAG chain similar to the one in the example and return the response of the `invoke` call with `query` as an argument
    INSERT CODE HERE

## Test out the tools

Test out the tools individually to make sure we are getting the right responses from them. Remember tools are LangChain Runnables so we can use the `invoke` method on them.


In [ ]:
get_paper_metadata_from_arxiv.invoke("Retrieval Augmented Generation")

In [ ]:
get_paper_summary_from_arxiv.invoke("1808.09236")

In [ ]:
get_paper_summary_from_arxiv.invoke("808.09236")

In [ ]:
answer_questions_about_topics.invoke("What are partial cubes?")

In [19]:
# Create the list of tools
tools = [
    get_paper_metadata_from_arxiv,
    get_paper_summary_from_arxiv,
    answer_questions_about_topics,
]

## Create a Basic Tool-calling Agent

Create a basic tool-calling agent using LangChain. Start with a basic prompt, then experiment with a Chain-of-Thought (prompt) if time permits.


In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools.render import render_text_description

In [ ]:
# Reference docs: https://api.python.langchain.com/en/latest/tools/langchain.tools.render.render_text_description.html
# Try out a simple prompt. Try to use the `render_text_description` method to include the list of tools the agent can access, in the prompt.
system_message = INSERT CODE HERE

In [ ]:
# CoT prompt -- Uncomment this system prompt if you want to try `CoT` prompting

# system_message = f"""Given a question, write out in a step-by-step manner your reasoning
# for how you will solve the problem to be sure that your conclusion is correct.
# Avoid simply stating the correct answer at the outset.You can answer directly if the user
# is greeting you or similar. Otherwise, you have access to the following tools:

# {render_text_description(tools)}

# Begin!
# """

In [ ]:
# Reference docs: https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/
# Refer to the `Create Agent` and `Run Agent` sections in the above docs to craft a prompt, initialize an agent, and an agent executor
# NOTE:
    # Use the `system_message` above as the system prompt
    # Do not include chat history in the prompt right now
    # Name the agent executor object `agent_executor`
INSERT CODE HERE

In [ ]:
# Test that the agent works as expected.
# If it runs into parsing errors, add appropriate arguments to the agent executor and try again.
# Reference docs: https://python.langchain.com/v0.1/docs/modules/agents/how_to/handle_parsing_errors/
agent_executor.invoke({"input": "Give me papers on the topic prompt compression."})

## **[BONUS]** Create a custom agent without using `create_tool_calling_agent`

Create a custom agent without using the `create_tool_calling_agent` abstraction.


In [22]:
# Reference docs: https://python.langchain.com/v0.1/docs/modules/agents/how_to/custom_agent/
# Refer to the docs above to build a custom agent without using the `create_tool_calling_agent` abstraction.
INSERT CODE HERE

## Create a ReAct Agent

Create an agent that uses ReAct prompting.


In [ ]:
# Pull a ready-made react prompt from LangChain hub -- Modify if needed
from langchain import hub

prompt = hub.pull("hwchase17/react")
prompt.pretty_print()

In [25]:
from langchain.agents import create_react_agent

# Reference docs: https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/
# Refer to the above docs to initialize a react agent and the agent executor
# NOTE:
    # `prompt` is already a prompt template, so no need to create your own
    # Do not include chat history in the prompt right now
    # Name the agent executor object `agent_executor`
INSERT CODE HERE

In [ ]:
# Test that the agent works as expected.
# If it runs into parsing errors, add appropriate arguments to the agent executor and try again.
# Reference docs: https://python.langchain.com/v0.1/docs/modules/agents/how_to/handle_parsing_errors/
agent_executor.invoke({"input": "Give me the summary for the paper 1808.09236."})

## Add Memory to Agents using MongoDB

Adding conversational message history to the agent. Message history is stored in and retrieved from a MongoDB collection.


In [27]:
from langchain_mongodb.chat_message_histories import MongoDBChatMessageHistory

def get_message_history(session_id: str) -> MongoDBChatMessageHistory:
    # Reference docs: https://python.langchain.com/v0.2/docs/integrations/memory/mongodb_chat_message_history/
    # Return an instance of `MongoDBChatMessageHistory`
    # Use variables defined previously for the `connection_string` and `database_name`
    # Choose a collection name of your choice to store the chat history
    INSERT CODE HERE

In [28]:
# Reference docs: https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/#create-agent
# Refer the above docs and modify the code below to include chat history
system_message = f"""Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherwise, you have access to the following tools:

{render_text_description(tools)}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        INSERT CODE HERE,
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [29]:
from langchain_core.runnables.history import RunnableWithMessageHistory

# Reference docs: https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/#langsmith
# Look at the example in the docs above to add message history to the agent.
agent_with_chat_history = INSERT CODE HERE

In [ ]:
# Test the agent with chat history to make sure it works
agent_with_chat_history.invoke(
    {"input": "Get me papers on Prompt Compression."},
    config={"configurable": {"session_id": "my-session"}},
)

In [ ]:
# Ask a follow-up question to make sure that the chat history is being used
agent_with_chat_history.invoke(
    {"input": "What is the title of the first paper you found?"},
    config={"configurable": {"session_id": "my-session"}},
)